In [12]:
#Dependencies
import pandas as pd
import hvplot.pandas
import numpy as np
import panel as pn
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats

# filtered_df DataFrame is already defined with the necessary data.

# Load the data
path = 'Averaged_Cleaned_filtered_data.csv'
Heart_disease_df = pd.read_csv(path)

filtered_df = Heart_disease_df

In [7]:
# Unique values for dropdowns and checkboxes
ethnicities = filtered_df['Ethnicity'].unique().tolist()
states = filtered_df['US States'].unique().tolist()
years = sorted(filtered_df['Year'].unique().astype(int).tolist())
age_ranges = filtered_df['Age range'].unique().tolist()
heart_disease_types = filtered_df['Heart Disease Type'].unique().tolist()

In [8]:
# Create interactive widgets

    # CheckBox Widgets
ethnicity_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]])
states_selector = pn.widgets.CheckBoxGroup(name='Select States', options=['All States'] + states, inline=True)
states_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select States', options=states, inline=True)

    # Drop Down Selectors
ethnicity_selector_dropdown = pn.widgets.Select(name='Select Ethnicity', options=ethnicities, value=ethnicities[0])

start_year_selector = pn.widgets.Select(name='Select Start Year', options=years, value=years[0])
end_year_selector = pn.widgets.Select(name='Select End Year', options=years, value=years[-1])

age_range_selector = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])
age_range_selector_dropdown = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])

heart_disease_type_selector = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])
heart_disease_type_selector_dropdown = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])

In [9]:
# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_checkbox, start_year_selector, end_year_selector, states_selector, age_range_selector, heart_disease_type_selector)
def update_plot(selected_ethnicities, start_year, end_year, selected_states, selected_age_range, selected_heart_disease_type):
    if not selected_ethnicities:
        return "Please select at least one ethnicity."
    
    # Determine if 'All States' is selected
    all_states_selected = 'All States' in selected_states
    if not selected_states or (not all_states_selected and not any(state in selected_states for state in states)):
        return "Please select at least one state."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    if not all_states_selected:
        filtered_data = filtered_data[filtered_df['US States'].isin(selected_states)]
    
    # Group by 'Ethnicity' and 'Year', then calculate the mean of 'Data_Value/100_000 People'
    grouped_data = filtered_data.groupby(['Ethnicity', 'Year'])['Data_Value/100_000 People'].mean().reset_index()
    
    # Create a line plot
    plot = grouped_data.hvplot.line(
        x='Year',
        y='Data_Value/100_000 People',
        by='Ethnicity',
        title=f'{selected_heart_disease_type} Mortality Rates by Ethnicity',
        ylabel='Average Data Value per 100,000 People',
        xlabel='Year',
        legend='top',
        width=800,
        height=600
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_checkbox, pn.Column(start_year_selector, end_year_selector)),
    pn.Row(states_selector),
    pn.Row(pn.Column(age_range_selector, heart_disease_type_selector)),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] Column
            [0] Select(options=[1999, 2000, 2001, ...], value=1999)
            [1] Select(options=[1999, 2000, 2001, ...], value=2019)
    [1] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['All States', ...])
    [2] Row
        [0] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)

In [10]:
# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_checkbox, start_year_selector, end_year_selector, states_selector, age_range_selector, heart_disease_type_selector)
def update_plot(selected_ethnicities, start_year, end_year, selected_states, selected_age_range, selected_heart_disease_type):
    if not selected_states:
        return "Please select at least one state."
    if not selected_ethnicities:
        return "Please select at least one ethnicity."
    
    # Determine if 'All States' is selected
    all_states_selected = 'All States' in selected_states
    if not all_states_selected and not any(state in selected_states for state in states):
        return "Please select at least one state."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    if not all_states_selected:
        filtered_data = filtered_data[filtered_df['US States'].isin(selected_states)]
    
    # Create a boxplot
    plot = filtered_data.hvplot.box(
        y='Data_Value/100_000 People',
        by=['Heart Disease Type', 'Age range', 'Ethnicity'],
        title=f'{selected_heart_disease_type} Mortality Rates Distribution by Ethnicity and Age Range',
        xlabel='Heart Disease Type, Age Range, Ethnicity',
        ylabel='Data Value per 100,000 People',
        legend='top_right',
        rot=45,
        xrotation=45,
        width=800,  # Set the width of the plot
        height=600  # Set the height of the plot
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_checkbox, pn.Column(start_year_selector, end_year_selector)),
    pn.Row(states_selector),
    pn.Row(pn.Column(age_range_selector, heart_disease_type_selector)),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] Column
            [0] Select(options=[1999, 2000, 2001, ...], value=1999)
            [1] Select(options=[1999, 2000, 2001, ...], value=2019)
    [1] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['All States', ...])
    [2] Row
        [0] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)

In [11]:
# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_dropdown, age_range_selector_dropdown, states_selector_checkbox, heart_disease_type_selector_dropdown)
def update_plot(selected_ethnicity, selected_age_range, selected_states, selected_heart_disease_type):
    if not selected_states:
        return "Please select at least one state."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'] == selected_ethnicity) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['US States'].isin(selected_states))
    ]
    
    if filtered_data.empty:
        return "No data available for the selected criteria."
    
    # Pivot the data to create a correlation matrix
    pivot_table = filtered_data.pivot_table(
        values='Data_Value/100_000 People', 
        index='Year', 
        columns='US States'
    )
    
    # Calculate the correlation matrix
    correlation_matrix = pivot_table.corr()

    # Create a heatmap
    heatmap = correlation_matrix.hvplot.heatmap(
        title=f'Correlation Matrix for {selected_heart_disease_type} Mortality Rates in {selected_ethnicity} ({selected_age_range})',
        cmap='viridis',
        width=800,
        height=600,
        rot=45,
        xrotation=45
    ).opts(colorbar=True)

    return heatmap

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_dropdown, pn.Column(age_range_selector_dropdown, heart_disease_type_selector_dropdown)),
    pn.Row(states_selector_checkbox),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] Select(options=['American Indian/Alaska N...], value='American Indian/Alaska N...)
        [1] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [1] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...])
    [2] ParamFunction(function, _pane=Markdown, defer_load=False)